In [1]:
import os
from os.path import join as oj
import sys
sys.path.insert(1, oj(sys.path[0], '..'))  # insert parent path



import torch
from torch.autograd import Variable
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader
from copy import deepcopy
import pickle as pkl
import pandas as pd

In [2]:
out_dir = '/scratch/users/vision/chandan/dl_theory/sweep_seed_and_hidden1' # differs for chandan/raaz

# pickle load example
fnames = os.listdir(out_dir)
results = [pd.Series(pkl.load(open(oj(out_dir, fname), "rb"))) for fname in fnames]
pd.concat(results, axis=1).T

,N,accs,batch_size,d_in,d_out,gamma_optimizer,hidden1,labs,losses,lr,max_acc,means,min_loss,norms,num_iters,num_layers,sds,seed,step_size_optimizer,weights
0,2000,"[44.1413, 55.21325, 66.55, 66.55, 66.55, 66.55...",2000,1,2,0.9,10,"[0, 1, 0]","[1098.4197998046875, 1015.0543212890625, 963.5...",1e-06,66.55,"[0, 20, 40]",869.082,"[[4.443533120052862, 0.6018111791065763], [4.4...",200,2,"[1, 1, 1]",2,1000,{0: {'2.weight': [[-0.30743137 -0.02144163 -0....
1,2000,"[65.888, 65.888, 65.888, 65.888, 65.93765, 65....",2000,1,2,0.9,4,"[0, 1, 0]","[1204.4912109375, 1195.8162841796875, 1186.959...",1e-06,66.3679,"[0, 20, 40]",938.612,"[[1.0922830297345252, 0.5687535798642251], [1....",200,2,"[1, 1, 1]",2,1000,{0: {'2.weight': [[ 0.11291571 -0.4426006 0....
2,2000,"[42.8742, 42.8742, 42.8742, 42.8742, 42.8742, ...",2000,1,2,0.9,2,"[0, 1, 0]","[1622.221435546875, 1621.997802734375, 1621.77...",1e-06,43.759,"[0, 20, 40]",1581.91,"[[0.7181743602473176, 0.7907956837816528], [0....",200,2,"[1, 1, 1]",1,1000,"{0: {'2.weight': [[-0.66565424 0.42420247], [..."
3,2000,"[55.50469999999999, 55.50469999999999, 55.5046...",2000,1,2,0.9,5,"[0, 1, 0]","[1156.06396484375, 1155.4306640625, 1154.78771...",1e-06,60.0453,"[0, 20, 40]",1045.48,"[[4.631121007794945, 0.8766859226721522], [4.6...",200,2,"[1, 1, 1]",3,1000,{0: {'2.weight': [[-0.04376592 0.26810282 -0....
4,2000,"[34.2, 34.2, 34.2, 34.2, 34.2, 34.2, 34.2, 34....",2000,1,2,0.9,1,"[0, 1, 0]","[2229.032958984375, 2228.851806640625, 2228.67...",1e-06,34.2,"[0, 20, 40]",2189.63,"[[0.4602914166202119, 1.5029032882422513], [0....",200,2,"[1, 1, 1]",1,1000,"{0: {'2.weight': [[-0.1938468], [0.46935427]],..."
5,2000,"[56.68339999999999, 56.4306, 56.162, 55.909200...",2000,1,2,0.9,6,"[0, 1, 0]","[1384.518310546875, 1326.81103515625, 1257.444...",1e-06,57.1732,"[0, 20, 40]",947.42,"[[2.377878394142101, 0.33280190132680687], [2....",200,2,"[1, 1, 1]",1,1000,{0: {'2.weight': [[ 0.14628874 -0.15890925 -0....
6,2000,"[44.3594, 44.3594, 55.1666, 65.8, 65.8, 65.8, ...",2000,1,2,0.9,4,"[0, 1, 0]","[1155.3450927734375, 1054.3829345703125, 980.9...",1e-06,65.8,"[0, 20, 40]",896.529,"[[2.2648886874102487, 0.10808812450035532], [2...",200,2,"[1, 1, 1]",1,1000,{0: {'2.weight': [[ 0.07466245 -0.06104977 0....
7,2000,"[37.57555, 37.5911, 37.5911, 37.6222, 37.6222,...",2000,1,2,0.9,4,"[0, 1, 0]","[1175.4937744140625, 1174.8306884765625, 1174....",1e-06,38.493,"[0, 20, 40]",1059.53,"[[3.994544219374802, 0.8906134989441377], [3.9...",200,2,"[1, 1, 1]",3,1000,{0: {'2.weight': [[ 0.09448344 0.38785964 -0....
8,2000,"[33.2, 33.2, 33.2, 33.2, 33.2, 33.2, 33.2, 33....",2000,1,2,0.9,1,"[0, 1, 0]","[1305.7435302734375, 1296.347900390625, 1287.0...",1e-06,66.8,"[0, 20, 40]",977.831,"[[0.027604210129376305, 2.825018837833568], [0...",200,2,"[1, 1, 1]",4,1000,"{0: {'2.weight': [[-0.8158289], [-0.581251]], ..."
9,2000,"[37.76215, 37.82435, 37.871, 37.9021, 37.99540...",2000,1,2,0.9,6,"[0, 1, 0]","[1018.5318603515625, 1017.770263671875, 1017.0...",1e-06,65.55,"[0, 20, 40]",908.613,"[[5.000003914797517, 1.0956303464409416], [4.9...",200,2,"[1, 1, 1]",3,1000,{0: {'2.weight': [[-0.28566983 -0.08025181 -0....
